In [19]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [20]:
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
import pandas as pd
from Utils.createURM import createBumpURM
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from sklearn.model_selection import ParameterSampler
import numpy as np

In [21]:
dataset = pd.read_csv("../../Input/interactions_and_impressions.csv")
URM = createBumpURM(dataset)

C:\Users\Mike\AppData\Local\Temp\ipykernel_23840\3845007784.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("../../Input/interactions_and_impressions.csv")


In [14]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM, train_percentage=0.80)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], verbose=False)

In [16]:
grid_size = 100
TUNE_ITER = 20
num_epochs = 5
worse_score = 0

init_param_grid = {'topK': [i for i in range(10, 400)],
                   'epochs': [i for i in range(10, 100)],
                   'lambda_i': [i for i in np.arange(0.00001, 0.01)],
                   'lambda_j': [i for i in np.arange(0.00001, 0.01)]
                   }

new_param_grid = init_param_grid.copy()
best_params_dict = {'score': worse_score, 'params': []}
tried_params_list = []

for epoch in range(num_epochs):

        # List of sampled hyperparameter combinations will be used for random search
        param_list = list(ParameterSampler(new_param_grid, n_iter=TUNE_ITER, random_state=0))

        # Searching the Best Parameters with Random Search
        rs_results_dict = {}
        for tune_iter in range(TUNE_ITER):
            # Get the set of parameter for this iteration
            strategy_params = param_list[tune_iter]

            recommender = SLIM_BPR_Cython(URM_train)
            recommender.fit(epochs=strategy_params['epochs'], topK=strategy_params['topK'], lambda_i=strategy_params['lambda_i'], lambda_j=strategy_params['lambda_j'])
            results, _ = evaluator_validation.evaluateRecommender(recommender)
            results = results.loc[10]['MAP']

            rs_results_dict[tuple(strategy_params.values())] = {'score': results}

            if results > best_params_dict['score']:
                best_params_dict['score'] = results
                best_params_dict['params'] = list(strategy_params.values())

        # Save the results in dataframe and sort it based on score
        df_rs_results = pd.DataFrame(rs_results_dict).T.reset_index()
        df_rs_results.columns = list(strategy_params.keys()) + ['score']
        df_rs_results = df_rs_results.sort_values(['score'], ascending=False).head(num_epochs - epoch)

        # If the best score from this epoch is worse than the best score,
        # then append the best hyperaparameters combination to this epoch dataframe
        if df_rs_results['score'].iloc[0] < best_params_dict['score']:
            new_row_dict = {}
            new_row_dict['score'] = best_params_dict['score']
            for idx, key in enumerate(init_param_grid):
                new_row_dict[key] = best_params_dict['params'][idx]

            df_rs_results = df_rs_results.append(new_row_dict, ignore_index=True)
            df_rs_results = df_rs_results.sort_values(['score'], ascending=False).head(num_epochs - epoch)

        print(df_rs_results.head(1).T.to_dict())

        # Get the worse and best hyperparameter combinations
        df_rs_results_min = df_rs_results[df_rs_results['score'] > worse_score].min(axis=0)
        df_rs_results_max = df_rs_results[df_rs_results['score'] > worse_score].max(axis=0)

        # Generate new hyperparameter space based on current worse and best hyperparameter combinations
        for key in init_param_grid:
            if isinstance(init_param_grid[key][0], int):
                new_param_grid[key] = np.unique(
                    [i for i in range(int(df_rs_results_min[key]), int(df_rs_results_max[key]) + 1)])
            elif isinstance(init_param_grid[key][0], float):
                new_param_grid[key] = np.unique(np.linspace(df_rs_results_min[key], df_rs_results_max[key], grid_size))
            else:
                new_param_grid[key] = init_param_grid[key]

        # Decrease the tuning iteration for random search
        TUNE_ITER = int(TUNE_ITER - epoch * TUNE_ITER / num_epochs)


URM_train, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage=0.80)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

recommender = SLIM_BPR_Cython(URM_train)
recommender.fit(epochs=best_params_dict['params'][1], topK=best_params_dict['params'][0], lambda_i=best_params_dict['params'][2], lambda_j=best_params_dict['params'][3])
result_df, _ = evaluator_test.evaluateRecommender(recommender)
result_df

Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.
Processed 41629 (100.0%) in 1.02 sec. BPR loss is 9.79E-07. Sample per second: 40766
SLIM_BPR_Recommender: Epoch 1 of 17. Elapsed time 0.22 sec
Processed 41629 (100.0%) in 0.25 sec. BPR loss is 2.96E-06. Sample per second: 167093
SLIM_BPR_Recommender: Epoch 2 of 17. Elapsed time 0.44 sec
Processed 41629 (100.0%) in 0.48 sec. BPR loss is 5.17E-06. Sample per second: 86696
SLIM_BPR_Recommender: Epoch 3 of 17. Elapsed time 0.68 sec
Processed 41629 (100.0%) in 0.71 sec. BPR loss is 7.55E-06. Sample per second: 58375
SLIM_BPR_Recommender: Epoch 4 of 17. Elapsed time 0.91 sec
Processed 41629 (100.0%) in 1.00 sec. BPR loss is 9.49E-06. Sample per second: 41756
SLIM_BPR_Recommender: Epoch 5 of 17. Elapsed time 1.19 sec
Processed 41629 (100.0%) in 1.25 sec. BPR loss is 1.12E-05.

C:\Users\Mike\AppData\Local\Temp\ipykernel_23840\2021437604.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rs_results = df_rs_results.append(new_row_dict, ignore_index=True)


{2: {'topK': 45.0, 'lambda_j': 75.0, 'lambda_i': 1e-05, 'epochs': 1e-05, 'score': 0.026702099357902197}}
Deallocating Cython objects
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.
Processed 41629 (100.0%) in 0.27 sec. BPR loss is 9.15E-07. Sample per second: 156310
SLIM_BPR_Recommender: Epoch 1 of 6. Elapsed time 0.25 sec
Processed 41629 (100.0%) in 0.50 sec. BPR loss is 2.87E-06. Sample per second: 83488
SLIM_BPR_Recommender: Epoch 2 of 6. Elapsed time 0.48 sec
Processed 41629 (100.0%) in 0.74 sec. BPR loss is 5.14E-06. Sample per second: 56484
SLIM_BPR_Recommender: Epoch 3 of 6. Elapsed time 0.72 sec
Processed 41629 (100.0%) in 0.97 sec. BPR loss is 6.87E-06. Sample per second: 42924
SLIM_BPR_Recommender: Epoch 4 of 6. Elapsed time 0.95 sec
Processed 41629 (100.0%) in 1.27 sec. BPR loss is 9.26E-06. Sample per sec

C:\Users\Mike\AppData\Local\Temp\ipykernel_23840\2021437604.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rs_results = df_rs_results.append(new_row_dict, ignore_index=True)


{1: {'topK': 45.0, 'lambda_j': 75.0, 'lambda_i': 1e-05, 'epochs': 1e-05, 'score': 0.026702099357902197}}
EvaluatorHoldout: Ignoring 321 ( 0.8%) Users that have less than 1 test interactions
Deallocating Cython objects
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.
Processed 41629 (100.0%) in 0.86 sec. BPR loss is 9.40E-07. Sample per second: 48398
SLIM_BPR_Recommender: Epoch 1 of 1e-05. Elapsed time 0.27 sec
SLIM_BPR_Recommender: Terminating at epoch 1. Elapsed time 23.14 sec
Deallocating Cython objects
EvaluatorHoldout: Processed 41308 (100.0%) in 23.70 sec. Users per second: 1743


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.046081,0.063365,0.055093,0.022578,0.03049,0.153802,0.068368,0.050185,0.31086,0.184846,...,0.992289,0.308463,0.992289,0.029697,8.035468,0.986046,0.058544,0.595571,5.326985,0.250837


In [17]:
import json

resultParameters = df_rs_results.to_json(orient="records")
parsed = json.loads(resultParameters)

with open("logs/" + recommender.RECOMMENDER_NAME + "_logs.json", 'w') as json_file:
    json.dump(parsed, json_file, indent=4)

In [18]:
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage=0.80)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

recommender = SLIM_BPR_Cython(URM_train)
recommender.fit(epochs=75, topK=45, lambda_i=1e-05, lambda_j=1e-05)
result_df, _ = evaluator_test.evaluateRecommender(recommender)
result_df

EvaluatorHoldout: Ignoring 328 ( 0.8%) Users that have less than 1 test interactions
Deallocating Cython objects
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.
Processed 41629 (100.0%) in 0.27 sec. BPR loss is 8.92E-07. Sample per second: 151495
SLIM_BPR_Recommender: Epoch 1 of 75. Elapsed time 0.21 sec
Processed 41629 (100.0%) in 0.51 sec. BPR loss is 2.86E-06. Sample per second: 81991
SLIM_BPR_Recommender: Epoch 2 of 75. Elapsed time 0.45 sec
Processed 41629 (100.0%) in 0.75 sec. BPR loss is 5.13E-06. Sample per second: 55148
SLIM_BPR_Recommender: Epoch 3 of 75. Elapsed time 0.69 sec
Processed 41629 (100.0%) in 0.99 sec. BPR loss is 7.04E-06. Sample per second: 41843
SLIM_BPR_Recommender: Epoch 4 of 75. Elapsed time 0.93 sec
Processed 41629 (100.0%) in 1.23 sec. BPR loss is 9.14E-06. Sample per second: 33932
SLIM_

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.053154,0.073309,0.063898,0.02654,0.036053,0.175264,0.078774,0.058033,0.347764,0.213539,...,0.992121,0.345024,0.992121,0.017449,7.628564,0.985018,0.034369,0.56537,5.615055,0.243462


In [22]:
from Utils.writeSubmission import write_submission

recommender = SLIM_BPR_Cython(URM)
recommender.fit(epochs=75, topK=45, lambda_i=1e-05, lambda_j=1e-05)

write_submission(recommender=recommender,
                 target_users_path="../Input/data_target_users_test.csv",
                 out_path='../Output/{}_submission.csv'.format('SLIMBPR'))

Deallocating Cython objects
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.
Processed 41629 (100.0%) in 0.70 sec. BPR loss is 2.06E-06. Sample per second: 59227
SLIM_BPR_Recommender: Epoch 1 of 75. Elapsed time 0.25 sec
Processed 41629 (100.0%) in 0.97 sec. BPR loss is 6.78E-06. Sample per second: 42747
SLIM_BPR_Recommender: Epoch 2 of 75. Elapsed time 0.52 sec
Processed 41629 (100.0%) in 1.26 sec. BPR loss is 1.11E-05. Sample per second: 33019
SLIM_BPR_Recommender: Epoch 3 of 75. Elapsed time 0.81 sec
Processed 41629 (100.0%) in 0.53 sec. BPR loss is 1.60E-05. Sample per second: 78561
SLIM_BPR_Recommender: Epoch 4 of 75. Elapsed time 1.08 sec
Processed 41629 (100.0%) in 0.81 sec. BPR loss is 2.27E-05. Sample per second: 51503
SLIM_BPR_Recommender: Epoch 5 of 75. Elapsed time 1.36 sec
Processed 41629 (100.0%) in 1.08

FileNotFoundError: [Errno 2] No such file or directory: '../Input/data_target_users_test.csv'